In [1]:
import openai
print("✅ OpenAI version:", openai.__version__)


✅ OpenAI version: 1.50.2


In [6]:
import sys
print(sys.executable)


/usr/local/bin/python3


In [3]:
from dotenv import load_dotenv
import os

# Load .env
load_dotenv()

# Get key and ensure it's visible to the environment
api_key = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = api_key  # 👈 ensures new SDK can auto-detect it

if api_key:
    print("✅ Key loaded and exported:", api_key[:12] + "..." + api_key[-5:])
else:
    raise ValueError("❌ No API key found — check your .env file")


✅ Key loaded and exported: sk-proj-dtvl...dV18A


In [6]:
from openai import OpenAI
client = OpenAI()  # now auto-detects the key
print("✅ Client initialized successfully.")


TypeError: Client.__init__() got an unexpected keyword argument 'proxies'

In [18]:
# =========================================================
# 1️⃣ Imports & Setup
# =========================================================
from dotenv import load_dotenv
import os, pandas as pd, numpy as np
from tqdm import tqdm
from openai import OpenAI
import random, json

# Load OpenAI Key from .env
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

if api_key:
    print("✅ Key loaded:", api_key[:12] + "..." + api_key[-5:])
else:
    raise ValueError("❌ No API key found. Please add OPENAI_API_KEY to your .env")

client = OpenAI(api_key=api_key)

# =========================================================
# 2️⃣ Load Dataset
# =========================================================
df = pd.read_csv("/Users/pulinkumar/Desktop/ALGOVERSE/Financial-QA-10k.csv")
df = df.fillna("")
print(f"✅ Dataset loaded: {df.shape}")
print(df.head(3))

# Just in case columns differ
text_col = "context"
question_col = "question"
answer_col = "answer"

# =========================================================
# 3️⃣ Generate Model Predictions (Baseline LLMs)
# =========================================================
# You can replace these with actual calls to local models (Mistral / LLaMA)
# For now, we simulate random model predictions
def mock_model_prediction(question):
    endings = [
        "increased financial stability.",
        "improved risk-adjusted returns.",
        "lowered capital costs.",
        "helped diversification efforts."
    ]
    return f"The company's strategy led to {random.choice(endings)}"

df["mistral_pred"] = df[question_col].apply(lambda q: mock_model_prediction(q))
df["llama_pred"] = df[question_col].apply(lambda q: mock_model_prediction(q))

print("✅ Mock model predictions added for Mistral & LLaMA.")

# =========================================================
# 4️⃣ Define the LLM-as-a-Judge function
# =========================================================
def llm_as_judge(question, true_answer, pred_answer, judge_model="gpt-4o-mini"):
    """
    Use ChatGPT (GPT-5/4o-mini) to semantically evaluate model prediction quality.
    """
    prompt = f"""
You are a financial QA evaluation assistant.

Question: {question}

Ground Truth Answer: {true_answer}
Predicted Answer: {pred_answer}

Evaluate the predicted answer from 0.0 to 1.0 based on:
1. **Correctness** – factual alignment with the ground truth.
2. **Completeness** – inclusion of key details.
3. **Relevance** – focus on answering the question.

Return only valid JSON:
{{"score": 0.xx, "justification": "short reasoning"}}
"""
    try:
        response = client.chat.completions.create(
            model=judge_model,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.2,
            max_tokens=200
        )
        content = response.choices[0].message.content.strip()
        result = json.loads(content)
        return result.get("score", 0.0), result.get("justification", "")
    except Exception as e:
        print("⚠️ Error:", e)
        return None, str(e)

# =========================================================
# 5️⃣ Evaluate Mistral & LLaMA Predictions
# =========================================================
results = []
judge_model = "gpt-4o-mini"  # or "gpt-5" if available

for model_name in ["mistral_pred", "llama_pred"]:
    print(f"\n🔎 Evaluating model: {model_name} with judge: {judge_model}")
    scores, reasons = [], []

    for _, row in tqdm(df.head(50).iterrows(), total=50):
        s, r = llm_as_judge(
            question=row[question_col],
            true_answer=row[answer_col],
            pred_answer=row[model_name],
            judge_model=judge_model
        )
        scores.append(s)
        reasons.append(r)

    df[f"{model_name}_score"] = scores
    df[f"{model_name}_reason"] = reasons
    avg_score = np.mean([s for s in scores if s is not None])
    print(f"✅ {model_name} → Average Judge Score: {avg_score:.3f}")

    results.append({"model": model_name, "avg_score": avg_score, "judge": judge_model})

# =========================================================
# 6️⃣ Save Results
# =========================================================
results_df = pd.DataFrame(results)
results_df.to_csv("/Users/pulinkumar/Desktop/ALGOVERSE/Financial-Agent-LLM-Judge-Results.csv", index=False)
df.to_csv("/Users/pulinkumar/Desktop/ALGOVERSE/Financial-Agent-LLM-Detailed.csv", index=False)
print("\n💾 Results saved to:")
print("  • Financial-Agent-LLM-Judge-Results.csv")
print("  • Financial-Agent-LLM-Detailed.csv")


✅ Key loaded: sk-proj-dtvl...dV18A


TypeError: Client.__init__() got an unexpected keyword argument 'proxies'